In [4]:
import numpy as np
import pandas as pd
import json
import os

In [5]:
with open("../../../../data/human_data/inference/experiment_1/out1.json", "r") as f:
    data = json.load(f)

In [6]:
data["demographics"]

{'gender': 'female',
 'age': '44',
 'race': 'asian',
 'fname': 'Shana',
 'date': '7/1/2019',
 'ethnicity': 'non hispanic'}

In [7]:
path = "../../../../data/human_data/inference/experiment_1/"
data_files = [file for file in os.listdir(path) if file.endswith(".json")]

In [8]:
data_files[0][3:-5]

'22'

In [9]:
demo_dict = {
    "condition": [],
    "participant": [],
    "age": [],
    "gender": [],
    "race": []
}

path = "../../../../data/human_data/inference/experiment_"
# data_files = [file for file in os.listdir(path) if file.endswith(".json")]

for i, exp in enumerate(["vision", "audio", "occluded"]):
    exp_path = path + str(i + 1) + "/"
    data_files = [file for file in os.listdir(exp_path) if file.endswith(".json")]

    for file in data_files:
        part_num = int(file[3:-5])
        with open(exp_path + file, "r") as f:
            data = json.load(f)

        part_demo = data["demographics"]
        demo_dict["condition"].append(exp)
        demo_dict["participant"].append(part_num)
        demo_dict["age"].append(part_demo["age"])
        demo_dict["gender"].append(part_demo["gender"])
        demo_dict["race"].append(part_demo["race"])


In [10]:
df_demo = pd.DataFrame(demo_dict)
df_demo["condition"] = pd.Categorical(df_demo["condition"], categories=["vision", "audio", "occluded"], ordered=True)
df_demo = df_demo.sort_values(by=["condition", "participant"])

error_row = df_demo["age"] == ""
df_demo.loc[error_row, ["age", "race"]] = df_demo.loc[error_row, ["race", "age"]].values

df_demo["age"] = df_demo["age"].astype(int)
df_demo["gender"] = df_demo["gender"].str.lower().str.strip()
df_demo["race"] = df_demo["race"].str.lower().str.strip()

In [11]:
df_demo

,condition,participant,age,gender,race
19,vision,1,44,female,asian
16,vision,2,22,female,asian
11,vision,3,21,male,asian
6,vision,4,21,male,mexican
3,vision,5,23,male,white
...,...,...,...,...,...
80,occluded,26,18,female,multiracial
82,occluded,27,18,male,mixed
70,occluded,28,18,female,black
77,occluded,29,18,female,white


In [12]:
mean_age = np.round(df_demo['age'].mean())
std_age = np.round(df_demo['age'].std())

mean_age, std_age

(24.0, 8.0)

In [15]:
gender_counts = df_demo['gender'].value_counts()
race_counts = df_demo['race'].value_counts()

gender_counts

gender
female    47
male      37
f          4
m          2
Name: count, dtype: int64

In [29]:
race_counts

race
white                           28
asian                           25
caucasian                        5
black                            2
south asian                      2
pacific islander                 2
chinese                          2
american                         1
chinese/white                    1
multiracial                      1
latina/chicana                   1
mixed, black and white           1
asian american                   1
asian indian                     1
mixed (japanese and british)     1
white/caucasian                  1
filipino                         1
african american/black           1
                                 1
aisan                            1
w                                1
european/southeast-asian         1
korean                           1
mexican                          1
indian (asian)                   1
ashia                            1
black/ african american          1
pakistani                        1
hispanic       

In [31]:
race_counts_dict = race_counts.to_dict()
race_counts_dict

race_consolidate = {
    "white": race_counts_dict["white"] + race_counts_dict["caucasian"] + race_counts_dict["white/caucasian"] + race_counts_dict["w"],
    "black": race_counts_dict["black"] + race_counts_dict["african american/black"] + race_counts_dict["black/ african american"],
    "asian": race_counts_dict["asian"] + race_counts_dict["south asian"] + race_counts_dict["chinese"] + race_counts_dict["asian american"] + race_counts_dict["asian indian"] + race_counts_dict["filipino"] + race_counts_dict["aisan"] + race_counts_dict["korean"] + race_counts_dict["indian (asian)"] + race_counts_dict["ashia"] + race_counts_dict["pakistani"] + race_counts_dict["east asian"],
    "american_indian": 0,
    "pacific_islander": race_counts_dict["pacific islander"],
    "multiracial": race_counts_dict["chinese/white"] + race_counts_dict["multiracial"] + race_counts_dict["mixed, black and white"] + race_counts_dict["mixed (japanese and british)"] + race_counts_dict["european/southeast-asian"] + race_counts_dict["mixed"],
    "other": race_counts_dict["american"] + race_counts_dict["latina/chicana"] + race_counts_dict[""] + race_counts_dict["mexican"] + race_counts_dict["hispanic"]
}

In [32]:
race_consolidate

{'white': 35,
 'black': 4,
 'asian': 38,
 'american_indian': 0,
 'pacific_islander': 2,
 'multiracial': 6,
 'other': 5}

In [33]:
np.sum(list(race_consolidate.values()))

90